# EstateWise Chapel Hill Real Estate Analysis & Gemini Chat CLI

Welcome to the EstateWise Colab notebook, where we combine advanced data science techniques with Google’s Gemini API to explore and interact with Chapel Hill’s real estate market. Whether you’re digging into trends, clustering similar homes, or simply chatting with our AI concierge, this notebook has you covered.

## This notebook covers

1. **Install dependencies**
2. **Mount Google Drive**
3. **Load Properties Data**
4. **Clean & wrangle data**
5. **Exploratory Data Analysis (EDA)**
6. **PCA / t-SNE + k-Means clustering**
7. **Cluster summaries**
8. **Interactive geospatial visualizations**
   - **Price heatmap** (density weighted by listing price)
   - **Living-area heatmap** (density weighted by square footage)
   - **Price-per-square-foot heatmap** (gradient showing per-unit value)
9. **Interactive Google Gemini CLI**
   - Agentic AI decides when to fetch property data
   - Five-expert ensemble with master-agent merge (MoE - Mixture of Experts Pipeline)
   - Type your questions & get tailored Chapel Hill home recommendations in real time!

<p align="center">
  <img src="https://img.shields.io/badge/Python-3.11-blue?style=flat-square&logo=python&logoColor=white" alt="Python"/>
  <img src="https://img.shields.io/badge/Jupyter-Notebook-orange?style=flat-square&logo=jupyter&logoColor=white" alt="Jupyter Notebook"/>
  <img src="https://img.shields.io/badge/pandas-Data%20Wrangling-150458?style=flat-square&logo=pandas&logoColor=white" alt="pandas"/>
  <img src="https://img.shields.io/badge/NumPy-Numerical%20Computing-013243?style=flat-square&logo=numpy&logoColor=white" alt="NumPy"/>
  <img src="https://img.shields.io/badge/scikit--learn-ML-F7931E?style=flat-square&logo=scikitlearn&logoColor=white" alt="scikit-learn"/>
  <img src="https://img.shields.io/badge/Matplotlib-Visualization-11557C?style=flat-square&logo=matplotlib&logoColor=white" alt="Matplotlib"/>
  <img src="https://img.shields.io/badge/Seaborn-Statistical%20Viz-76B5C5?style=flat-square&logo=seaborn&logoColor=white" alt="Seaborn"/>
  <img src="https://img.shields.io/badge/Folium-Mapping-00A5CF?style=flat-square&logo=mapbox&logoColor=white" alt="Folium"/>
  <img src="https://img.shields.io/badge/Pinecone-Vector%20DB-663399?style=flat-square&logo=pinecone&logoColor=white" alt="Pinecone"/>
  <img src="https://img.shields.io/badge/Google%20Gemini-AI-4285F4?style=flat-square&logo=google&logoColor=white" alt="Google Gemini"/>
  <img src="https://img.shields.io/badge/python--dotenv-Configuration-3C873A?style=flat-square&logo=dotenv&logoColor=white" alt="python-dotenv"/>
</p>

> To run the code in this notebook, download the notebook and then upload it to [Google Colab](https://colab.research.google.com). Alternatively, go straight to [this Google Colab Notebook link](https://colab.research.google.com/drive/1-Z3h0LUHl0v-e0RaZgwruL8q180Uk4Z-?usp=sharing) to view & run it directly in Google Colab.

> Before running the notebook's code, ensure that you have loaded the datasets to your Google Drive. If you prefer to not change any code, put your datasets (e.g. `Zillow-March2025-dataset_part0.json`, `Zillow-March2025-dataset_part1.json`, `Zillow-March2025-dataset_part2.json`, `Zillow-March2025-dataset_part3.json`) in the `COMP-488` folder in your Drive.

> Note: If you plan to use all dataset files with this notebook (over 30,000 properties), make sure to select the v2-8 TPU runtime or higher. Otherwise, Colab may crash due to the large dataset size. This notebook has been successfully run on all files, handling more than 30,000 properties.

# 1. Install Depenencies & Setup

In [ ]:
# 1. Install Dependencies
!pip install --upgrade pandas numpy scikit-learn matplotlib seaborn google-generativeai python-dotenv

# 2. Imports & Mount Google Drive

In [ ]:
# 2. Imports & Mount Drive
import os, glob, json, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Google Gemini client
import google.generativeai as genai
from dotenv import load_dotenv

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

### 2.1 Configure Paths & API Key

- Point `path488` to your Zillow JSON folder in Drive
- Set `GOOGLE_API_KEY` in Colab’s environment (Settings → Secrets)
- Set `PINECONE_API_KEY` in Colab’s environment (Settings → Secrets)
- Set `PINECONE_ENVIRONMENT` in Colab’s environment (Settings → Secrets)
- Set `PINECONE_INDEX` in Colab’s environment (Settings → Secrets)

In [ ]:
# Path to folder containing your Zillow JSONs
path488 = '/content/drive/MyDrive/COMP-488/'
zillowfiles = glob.glob(path488 + 'Zillow*.json')
# Keep only 2025 files
zillow25 = [f for f in zillowfiles if '2025' in f]
print("Found 2025 JSON files:", zillow25)

## 3. Load & Concatenate 2025 Data

In [ ]:
dfs = []
for fp in zillow25:
    with open(fp, 'r', encoding='utf8') as f:
        arr = json.load(f)
    dfs.append(pd.DataFrame(arr))
df_raw = pd.concat(dfs, ignore_index=True)
print(f"Raw 2025 records: {len(df_raw):,}")

## 4. Data Cleaning & Wrangling

- Keep only relevant fields
- Apply safe parsing and drop invalid entries

In [ ]:
# 4.1 Helpers & field list
def safe_str(val, fallback="Unknown"):
    return val.strip() if isinstance(val, str) and val.strip() else fallback

def safe_num(val, fallback=0, mn=None, mx=None):
    try:
        n = float(val)
    except:
        return fallback
    if mn is not None and n < mn:
        return fallback
    if mx is not None and n > mx:
        return fallback
    return n

keep = [
    "zpid","address","city","state","bedrooms","bathrooms",
    "price","yearBuilt","livingArea","latitude","longitude",
    "homeType","listingDataSource","description"
]

In [ ]:
# 4.2 Clean records
import pandas as pd
import numpy as np
import json

# Helper functions
def safe_num(val, default=0, min_val=None, max_val=None):
    """
    Convert a value to float, applying bounds and default on failure or missing.
    """
    try:
        if pd.isna(val):
            return default
    except:
        pass
    try:
        # strip commas, whitespace
        num = float(str(val).replace(",", "").strip())
    except:
        return default
    if (min_val is not None and num < min_val) or (max_val is not None and num > max_val):
        return default
    return num

def safe_int(val, default=0, min_val=None, max_val=None):
    """
    Convert to int safely via safe_num, then cast.
    """
    num = safe_num(val, default, min_val, max_val)
    return int(round(num))

def safe_float(val, default=0.0, min_val=None, max_val=None, step=0.5):
    """
    Convert to float safely via safe_num, then round to nearest step (e.g. 0.5 for bathrooms).
    """
    num = safe_num(val, default, min_val, max_val)
    # round to nearest step
    return round(num / step) * step

def safe_str(val, default="Unknown"):
    """
    Ensure string, JSON-serialize dicts/lists, default if missing/empty.
    """
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return default
    if isinstance(val, (dict, list)):
        try:
            return json.dumps(val)
        except:
            return default
    s = str(val).strip()
    return s if s else default

# 4.3 Clean records (improved)
current_year = pd.Timestamp.now().year
records = []

for r in df_raw[keep].to_dict('records'):
    # normalize address field if it's a JSON string
    addr_field = r.get("address")
    if isinstance(addr_field, str):
        try:
            addr = json.loads(addr_field)
        except:
            addr = {}
    else:
        addr = addr_field or {}

    # safe extraction
    zpid = safe_int(r.get("zpid"), default=0)
    street = safe_str(addr.get("streetAddress"), default="")
    city   = safe_str(addr.get("city") or r.get("city"), default="")
    state  = safe_str(addr.get("state") or r.get("state"), default="")
    zipcode= safe_str(addr.get("zipcode"), default="")

    # skip if essential address components missing
    if zpid == 0 or not street or not city or not state or not zipcode:
        continue

    # numeric fields
    bedrooms  = safe_int(r.get("bedrooms"), default=0, min_val=0, max_val=20)
    bathrooms = safe_float(r.get("bathrooms"), default=0.0, min_val=0, max_val=20, step=0.5)
    price     = safe_num(r.get("price"), default=0, min_val=10000, max_val=1e8)

    # handle yearBuilt bounds
    year_val = safe_int(r.get("yearBuilt"), default=0)
    if year_val < 1800 or year_val > current_year + 1:
        yearBuilt = 0
    else:
        yearBuilt = year_val

    livingArea = safe_num(r.get("livingArea"), default=0, min_val=100, max_val=20000)

    # geolocation
    latitude  = safe_num(r.get("latitude"), default=0.0, min_val=-90, max_val=90)
    longitude = safe_num(r.get("longitude"), default=0.0, min_val=-180, max_val=180)

    # categorical/text fields
    homeType           = safe_str(r.get("homeType"))
    listingDataSource  = safe_str(r.get("listingDataSource"))
    description        = safe_str(r.get("description"), default="No description provided.")

    records.append({
        "zpid": zpid,
        "street": street,
        "city": city,
        "state": state,
        "zipcode": zipcode,
        "bedrooms": bedrooms,
        "bathrooms": bathrooms,
        "price": price,
        "yearBuilt": yearBuilt,
        "livingArea": livingArea,
        "latitude": latitude,
        "longitude": longitude,
        "homeType": homeType,
        "listingDataSource": listingDataSource,
        "description": description,
    })

# create DataFrame
df = pd.DataFrame(records)
print(f"Cleaned records: {len(df):,}")
df.head()


## 5. Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

# --- 5.1 Overview ---
print(df.dtypes, "\n")
print("Missing values:\n", df.isna().sum(), "\n")

# --- 5.2 Summary stats ---
display(df.describe().T)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

df_filtered = df[df["price"].between(0, 20_000_000)]

# --- 5.3 Price Distribution (Original) ---
fig, ax = plt.subplots(figsize=(6, 4))
sns.histplot(df["price"], bins=50, ax=ax)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

ax.tick_params(axis='x', labelrotation=45)
plt.setp(ax.get_xticklabels(), ha='right')

ax.set_title("5.3 Price Distribution (All Prices)")
ax.set_xlabel("Price (USD)")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()


# --- 5.3 Price Distribution (0–20 M USD) ---
fig, ax = plt.subplots(figsize=(6, 4))
sns.histplot(df_filtered["price"], bins=50, ax=ax)
ax.set_xlim(0, 20_000_000)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

ax.tick_params(axis='x', labelrotation=45)
plt.setp(ax.get_xticklabels(), ha='right')

ax.set_title("5.3 Price Distribution (0–20 M USD)")
ax.set_xlabel("Price (USD)")
ax.set_ylabel("Count")
plt.tight_layout()
plt.show()


# --- 5.4 Price vs Living Area (Original) ---
fig, ax = plt.subplots(figsize=(6, 4))
sns.scatterplot(x="livingArea", y="price", data=df, alpha=0.4, ax=ax)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

ax.tick_params(axis='x', labelrotation=45)
plt.setp(ax.get_xticklabels(), ha='right')

ax.set_title("5.4 Price vs Living Area (All Prices)")
ax.set_xlabel("Living Area (sqft)")
ax.set_ylabel("Price (USD)")
plt.tight_layout()
plt.show()


# --- 5.4 Price vs Living Area (0–20 M USD) ---
fig, ax = plt.subplots(figsize=(6, 4))
sns.scatterplot(x="livingArea", y="price", data=df_filtered, alpha=0.4, ax=ax)
ax.set_ylim(0, 20_000_000)
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

ax.tick_params(axis='x', labelrotation=45)
plt.setp(ax.get_xticklabels(), ha='right')

ax.set_title("5.4 Price vs Living Area (0–20 M USD)")
ax.set_xlabel("Living Area (sqft)")
ax.set_ylabel("Price (USD)")
plt.tight_layout()
plt.show()


In [ ]:
if 'address' not in df.columns:
    def make_address(row):
        street = row.get('streetAddress', '') or ''
        city   = row.get('city', '') or ''
        state  = row.get('state', '') or ''
        zipc   = row.get('zipcode', '') or ''
        parts = [street, city, state, zipc]
        return ", ".join([p for p in parts if p])
    df['address'] = df.apply(make_address, axis=1)

# 5.5 Bedrooms distribution
plt.figure(figsize=(6, 4))
sns.countplot(x="bedrooms", data=df)
plt.title("Distribution of Number of Bedrooms")
plt.xlabel("Bedrooms")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 5.6 Bathrooms distribution
plt.figure(figsize=(6, 4))
sns.countplot(x="bathrooms", data=df)
plt.title("Distribution of Number of Bathrooms")
plt.xlabel("Bathrooms")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 5.7 Year Built distribution
plt.figure(figsize=(8, 4))
plt.hist(df["yearBuilt"].dropna(), bins=30)
plt.title("Year Built Distribution")
plt.xlabel("Year Built")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 5.8 Correlation matrix heatmap
corr = df[["price", "livingArea", "bedrooms", "bathrooms", "yearBuilt"]].corr()
plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()

# 5.9 Boxplot of Price by Home Type
plt.figure(figsize=(8, 4))
sns.boxplot(x="homeType", y="price", data=df)
plt.title("Price by Home Type")
plt.xlabel("Home Type")
plt.ylabel("Price (USD)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 5.10 Top 10 most expensive properties
top10 = df.nlargest(10, "price")[["address", "price", "bedrooms", "bathrooms", "livingArea", "yearBuilt", "homeType"]]
print("Top 10 Most Expensive Properties:\n", top10.to_string(index=False))

## 6. PCA, t-SNE & k‑Means Clustering

In [ ]:
# 6.1 Impute & Normalize features
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

features = ["price", "bedrooms", "bathrooms", "livingArea", "yearBuilt"]

# 1) Impute missing values with column means
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(df[features])

# 2) Scale to [0,1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X_imputed)

# 6.2 PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=42)
df["pca1"], df["pca2"] = pca.fit_transform(X).T

# 6.3 t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42, perplexity=50)
df["tsne1"], df["tsne2"] = tsne.fit_transform(X).T

# 6.4 k-Means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X)

# Visualize clusters
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,5))
sns.scatterplot(x="pca1", y="pca2", hue="cluster", data=df, palette="tab10", s=10)
plt.title("PCA Projection by Cluster")
plt.show()

plt.figure(figsize=(6,6))
sns.scatterplot(x="tsne1", y="tsne2", hue="cluster", data=df, palette="tab10", s=10)
plt.title("t-SNE Projection by Cluster")
plt.show()


## 7. Cluster Summaries

In [ ]:
cluster_summary = df.groupby("cluster").agg(
    count=("zpid","size"),
    avg_price=("price","mean"),
    avg_sqft=("livingArea","mean"),
    avg_year=("yearBuilt","mean")
).round(2).reset_index()
cluster_summary

## 8. Interactive Geospatial Visualization of Listings

In this section we’ll build three different interactive maps in Colab to explore our cleaned and clustered real estate listings:

1. **Clustered Circle Markers with Layers**
   - Centered on Chapel Hill, NC
   - Fullscreen, measurement tool, and minimap controls
   - One FeatureGroup per cluster (Cluster 0…Cluster 3), toggleable via the layer control
   - Circle markers colored by cluster, with popups showing address, price, beds/baths, living area, year built

2. **Home-Type Map with Search Bar**
   - One FeatureGroup per `homeType` (e.g. Single Family, Condo)
   - Search plugin to lookup any property by its full address
   - Toggle layers for each home type via the layer control
   - Popups and tooltips include type, price, and basic specs

3. **Count-based MarkerCluster**
   - MarkerCluster with custom div icons that display the number of listings in each cluster
   - No heatmap—clusters simply show counts, and clicking expands individual markers with popups
   - Layer control to toggle the count-cluster overlay

Each map leverages **Folium** and supports intuitive pan/zoom, layer toggling, and rich popups/tooltips to help you visually explore spatial patterns in Chapel Hill real estate data. Simply run the corresponding code cells below to render each interactive map in your notebook.


In [ ]:
pip install folium

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap, MiniMap, Fullscreen, MeasureControl

# -------------------------------------------------------------------
# 8. Interactive Geospatial Visualization of Listings
# -------------------------------------------------------------------

# Define chunked‐loading options so markers are added in small, non-blocking batches
cluster_opts = {
    'chunkedLoading': True,
    'chunkInterval': 100,   # ms of work per chunk
    'chunkDelay': 5         # ms pause between chunks
}

# 8.1 Center map on Chapel Hill, NC
chapel_hill_coords = [35.9132, -79.0558]
m = folium.Map(
    location=chapel_hill_coords,
    zoom_start=13,
    tiles='CartoDB positron',
    control_scale=True
)

# 8.2 Add Fullscreen and Measure controls
Fullscreen(position='topright').add_to(m)
MeasureControl(position='topright', primary_length_unit='feet').add_to(m)

# 8.3 Add a minimap inset
MiniMap(toggle_display=True, position='bottomright').add_to(m)

# 8.4 Prepare a color palette for clusters
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige']
cluster_colors = {c: colors[c % len(colors)] for c in sorted(df['cluster'].unique())}

# 8.5 Create one hidden FeatureGroup + MarkerCluster for all properties
cluster_fg = folium.FeatureGroup(name='Property Clusters', show=False)
m.add_child(cluster_fg)
marker_cluster = MarkerCluster(
    name='Property Clusters',
    options={
        **cluster_opts,
        'spiderfyOnMaxZoom': False,
        'disableClusteringAtZoom': 17,
        'maxClusterRadius': 35
    }
).add_to(cluster_fg)

# 8.6 Add all listings into the single MarkerCluster
for _, row in df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    if pd.isna(lat) or pd.isna(lon):
        continue

    c = row['cluster']
    color = cluster_colors[c]

    popup_html = (
        f"<strong>{row['street']}, {row['city']}</strong><br>"
        f"Price: ${row['price']:,}<br>"
        f"Beds: {row['bedrooms']}, Baths: {row['bathrooms']}<br>"
        f"Area: {row['livingArea']} sqft, Year: {row['yearBuilt']}<br>"
        f"Cluster: {c}"
    )
    tooltip = f"${row['price']:,} — {row['bedrooms']}bd/{row['bathrooms']}ba"

    folium.CircleMarker(
        location=(lat, lon),
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.5,
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=tooltip
    ).add_to(marker_cluster)

# 8.7 Price Heatmap layer (visible by default)
heat_fg = folium.FeatureGroup(name='Price Heatmap', show=True)
m.add_child(heat_fg)
heat_data = df[['latitude', 'longitude', 'price']].dropna().values.tolist()
HeatMap(
    data=heat_data,
    name='Price Heatmap',
    min_opacity=0.3,
    radius=8,
    blur=5,
    max_zoom=1
).add_to(heat_fg)

# 8.8 Layer control (clusters unchecked, heatmap checked)
folium.LayerControl(collapsed=False, position='topright').add_to(m)

# Display the map
m


### 8.9 Price Heatmap Guide & Interpretation

#### 8.9.1. Guide

The heatmap layer overlays a semi‐transparent “intensity” map on top of the base map. It encodes **both spatial density** and **price weight** of the listings:

- **Color gradient**
  - **Cool colors (blue)** represent areas with relatively **few** or **lower‐priced** listings.
  - **Warm colors (yellow → red)** indicate areas with **many** and/or **higher‐priced** listings.

- **Intensity / Opacity**
  - Brighter, more opaque spots show where high‐price listings cluster closely together.
  - Fainter, more transparent spots show sparser or lower‐price concentrations.

- **Radius & Blur settings**
  - Each listing contributes to a circular “footprint” (radius = 8 px) that is blurred (blur = 5 px) to create a smooth gradient.
  - A larger radius or blur makes hot spots spread out, while smaller values concentrate them tightly around each point.

- **Weighting by Price**
  - Because we passed `[lat, lon, price]` as each data point, pricier listings contribute more to the heatmap intensity than cheaper ones.
  - Areas with both high listing counts *and* high prices will therefore appear hottest (deep red).

Use this layer to quickly identify neighborhoods where expensive homes are concentrated versus those dominated by lower‐priced or fewer listings. You can also toggle the clusters on/off in the layer control at the top right.

### 8.9.2 Interpretation

This layer shows relative listing‐price density across Chapel Hill, using a blue→green→red gradient (blue = low, green = mid, red = high).

- **Greenish areas near UNC (e.g. E Franklin St.)**
  These are mid-to-high price pockets—reflecting student rentals, townhomes and smaller single-family homes that command steady demand but aren’t the priciest in town.

- **Localized red hotspots within those green zones**
  Indicate clusters of premium listings (large homes or newly built properties) fetching the highest prices per square foot, even among an already expensive corridor.

- **Wider green bands in central Chapel Hill**
  Show established residential neighborhoods with moderate price levels—balanced value between proximity to amenities and home size.

- **Blue-ish outskirts**
  Reveal more affordable sectors—older subdivisions or edge-of-town areas where average listing prices drop off, offering entry-level options further from the center.

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MeasureControl

# -------------------------------------------------------------------
# 8.10 Interactive Geospatial Visualization of Living Area
# -------------------------------------------------------------------

# 8.10 Center map on Chapel Hill, NC
living_area_map = folium.Map(
    location=chapel_hill_coords,     # reuse coords from previous map
    zoom_start=13,
    tiles='CartoDB positron',
    control_scale=True
)

# 8.11 Add Fullscreen and Measure controls
Fullscreen(position='topright').add_to(living_area_map)
MeasureControl(position='topright', primary_length_unit='feet').add_to(living_area_map)

# 8.12 Add a minimap inset
MiniMap(toggle_display=True, position='bottomright').add_to(living_area_map)

# 8.13 Living Area Heatmap layer (density weighted by livingArea)
living_area_data = df[['latitude', 'longitude', 'livingArea']].dropna().values.tolist()
HeatMap(
    data=living_area_data,
    name='Living Area Heatmap',
    min_opacity=0.3,
    radius=8,
    blur=5,
    max_zoom=1
).add_to(living_area_map)

# 8.14 Layer control (heatmap visible by default)
folium.LayerControl(collapsed=False, position='topright').add_to(living_area_map)

# Display the map
living_area_map


### 8.15 Living‐Area Heatmap Guide & Interpretation

#### 8.15.1 Guide

The living‐area heatmap overlays a semi-transparent “intensity” layer on the base map, encoding both **spatial density** and **square-footage weight** of each listing:

- **Color gradient**
  - **Cool colors (blue)** represent areas with relatively **few** or **smaller-sized** listings.
  - **Warm colors (yellow → red)** indicate areas with **many** and/or **larger** homes.

- **Intensity / Opacity**
  - Brighter, more opaque spots show where large-square-foot listings cluster tightly.
  - Fainter, more transparent areas indicate sparser or smaller-sized concentrations.

- **Radius & Blur settings**
  - Each listing contributes a circular footprint (radius = 8 px) that’s blurred (blur = 5 px) for smooth transitions.
  - Smaller radii concentrate heat around individual points; larger radii spread it out.

- **Weighting by Living Area**
  - Because we passed `[lat, lon, livingArea]` into the heatmap, listings with more square footage contribute more to the intensity than compact ones.
  - Neighborhoods featuring both high listing counts **and** large homes will therefore appear hottest (deep red).

Use this layer to spot where the biggest homes are clustered versus areas dominated by smaller dwellings. You can toggle other layers (cluster markers, price heatmap, etc.) on and off via the layer control.

#### 8.15.2 Interpretation

- **Widespread green around UNC and central Chapel Hill**
  Indicates a high density of mid-sized homes—common student rentals, townhouses, and modest single-family houses close to campus.

- **Red-tinged “dots” within those green zones**
  Mark pockets of very large homes or new luxury developments that exceed the surrounding average square footage.

- **Shrinking green clusters with distance**
  Shows that as you move farther from the center, fewer large-footprint homes are listed, and overall home sizes trend smaller.

By comparing this living‐area layer with the price or price-per-sqft maps, you can decide whether larger homes command proportional prices or represent undervalued opportunities in certain neighborhoods.


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MeasureControl

# -------------------------------------------------------------------
# 8.16 Interactive Geospatial Visualization of Price per Square Foot
# -------------------------------------------------------------------

# 8.16 Center map on Chapel Hill, NC
price_psf_map = folium.Map(
    location=chapel_hill_coords,     # reuse coords from previous maps
    zoom_start=13,
    tiles='CartoDB positron',
    control_scale=True
)

# 8.17 Add Fullscreen and Measure controls
Fullscreen(position='topright').add_to(price_psf_map)
MeasureControl(position='topright', primary_length_unit='feet').add_to(price_psf_map)

# 8.18 Add a minimap inset
MiniMap(toggle_display=True, position='bottomright').add_to(price_psf_map)

# 8.19 Compute price per square foot and prepare heatmap data
df['price_per_sqft'] = df['price'] / df['livingArea']
price_psf_data = df[['latitude', 'longitude', 'price_per_sqft']].dropna().values.tolist()

# 8.20 Price-per-sqft Heatmap layer (visible by default)
# Note: gradient keys must be strings so folium doesn’t try to camelCase them
HeatMap(
    data=price_psf_data,
    name='Price per Sqft Heatmap',
    min_opacity=0.3,
    radius=8,
    blur=5,
    max_zoom=1,
    gradient={
        '0.2': 'blue',
        '0.4': 'lime',
        '0.6': 'yellow',
        '0.8': 'orange',
        '1.0': 'red'
    }
).add_to(price_psf_map)

# 8.21 Layer control (heatmap visible by default)
folium.LayerControl(collapsed=False, position='topright').add_to(price_psf_map)

# Display the map
price_psf_map


### 8.22 Interpretation of Price per Square Foot Heatmap

This heatmap highlights the spatial variation in value — how much each square foot of living space costs across Chapel Hill.

- **Red “hot spots”** show neighborhoods where you pay a premium per square foot, often corresponding to historically desirable areas close to downtown or the UNC campus.
- **Blue “cooler zones”** indicate more affordable pockets where larger homes may be available at lower per-square-foot rates, such as newer subdivisions on the outskirts.
- **Transitional corridors** (green-to-yellow bands) along major roads suggest mixed price points—areas where mid-range properties and townhomes create a broader spread of values.
- **Comparison with raw price map**: A neighborhood might have high overall prices but only moderate price-per-sqft (large lot sizes), or vice versa (smaller homes driving up per-sqft cost), helping you discern true value versus absolute price.
- **Investment insight**: Cooler areas with rising price-per-sqft trends may signal emerging markets, while established hotspots warrant premium budgets.


In [ ]:
import pandas as pd
import folium
import numpy as np
from branca.colormap import StepColormap
from folium.plugins import MiniMap, Fullscreen, MeasureControl

# -------------------------------------------------------------------
# 8.33 Interactive Geospatial Visualization of Grid-Based Composite Score
# -------------------------------------------------------------------

# 8.33 Center map on Chapel Hill, NC
grid_score_map = folium.Map(
    location=chapel_hill_coords,  # reuse coords
    zoom_start=13,
    tiles='CartoDB positron',
    control_scale=True
)

# 8.34 Add Fullscreen and Measure controls
Fullscreen(position='topright').add_to(grid_score_map)
MeasureControl(position='topright', primary_length_unit='feet').add_to(grid_score_map)

# 8.35 Add a minimap inset
MiniMap(toggle_display=True, position='bottomright').add_to(grid_score_map)

# 8.36 Compute price_per_sqft if not already present
if 'price_per_sqft' not in df:
    df['price_per_sqft'] = df['price'] / df['livingArea']

# 8.37 Define a regular lat/lon grid (~0.005° ≈ 500 m cells)
lat_min, lat_max = df['latitude'].min(), df['latitude'].max()
lon_min, lon_max = df['longitude'].min(), df['longitude'].max()
grid_size = 0.005
lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)

df['lat_bin'] = pd.cut(
    df['latitude'],
    bins=lat_bins,
    include_lowest=True,
    labels=lat_bins[:-1]
)
df['lon_bin'] = pd.cut(
    df['longitude'],
    bins=lon_bins,
    include_lowest=True,
    labels=lon_bins[:-1]
)

# 8.38 Aggregate metrics per grid cell
grid = (
    df
    .dropna(subset=['lat_bin', 'lon_bin'])
    .groupby(['lat_bin', 'lon_bin'], observed=False)
    .agg(
        count=('price', 'count'),
        avg_psf=('price_per_sqft', 'mean'),
        avg_area=('livingArea', 'mean'),
        avg_beds=('bedrooms', 'mean')
    )
    .reset_index()
)

grid = grid[grid['count'] >= 3]

# 8.39 Compute percentile ranks (0–1) for each metric
for col in ['avg_psf', 'avg_area', 'avg_beds']:
    grid[col + '_pct'] = grid[col].rank(pct=True, method='average')

# 8.40 Compute composite score (mean percentile × 100)
grid['score'] = grid[['avg_psf_pct', 'avg_area_pct', 'avg_beds_pct']].mean(axis=1) * 100

# 8.41 Drop NaN scores
grid = grid.dropna(subset=['score'])

# 8.42 Prepare a stepped colormap
colormap = StepColormap(
    colors=['green', 'yellow', 'orange', 'red'],
    index=[0, 25, 50, 75, 100],
    vmin=0, vmax=100,
    caption='Grid Composite Score (Percentile-Based)'
)
colormap.add_to(grid_score_map)

# 8.43 Plot a label at each cell’s centroid showing its score
for _, row in grid.iterrows():
    lat_ctr = float(row['lat_bin']) + grid_size / 2
    lon_ctr = float(row['lon_bin']) + grid_size / 2
    sc = row['score']
    color = colormap(sc)
    folium.map.Marker(
        location=(lat_ctr, lon_ctr),
        icon=folium.DivIcon(
            html=(
                f'<div style="background:{color}; '
                'border-radius:50%; width:24px; height:24px; '
                'line-height:24px; text-align:center; '
                'font-size:10px; color:white; font-weight:bold;">'
                f'{int(round(sc))}</div>'
            )
        )
    ).add_to(grid_score_map)

# 8.44 Layer control
folium.LayerControl(collapsed=False, position='topright').add_to(grid_score_map)

# Display the map
grid_score_map


### 8.45 Interpretation of Percentile-Based Composite Score Map

This grid-based map uses percentile ranking to show how each ~500 m cell stacks up relative to all others on a combined index of average price / sq ft, living area, and bedroom count. Scores run from 0 (lowest overall) to 100 (highest overall).

- **Red cells (scores ≥ 75)**
  These are the top-quartile areas in Chapel Hill, where homes tend to command the highest per-square-foot rates, offer larger living spaces, and include more bedrooms compared to most other cells. They typically correspond to established, high-demand neighborhoods close to UNC and downtown.

- **Orange-to-yellow cells (scores 25 – 74)**
  Occupying the middle two quartiles, these zones feature moderate combinations of value, size, and bedroom count. You’ll find mixed-price developments, suburban corridors, and transitional neighborhoods here—areas with solid amenities but fewer premium attributes.

- **Green cells (scores < 25)**
  Represent the bottom-quartile cells, where either smaller homes, lower price / sq ft, or fewer bedrooms dominate. These are often more affordable pockets, older subdivisions, or outlying sectors of Chapel Hill.

- **Percentile-based scaling** allows you to see not just absolute metrics but relative performance—ideal for quickly spotting outlier grids (both premium enclaves and value opportunities) without being skewed by extreme outliers in price or size.

### 8.46 Explanation of Composite Score Factors & Calculation

The composite score for each ~500 m grid cell is built from three equally‐weighted property metrics:

1. **Average Price per Square Foot (`avg_psf`)**
   - Captures how much you pay for living space in that cell.
2. **Average Living Area (`avg_area`)**
   - Reflects the typical home size (in square feet) within the cell.
3. **Average Bedroom Count (`avg_beds`)**
   - Measures how many bedrooms the average listing offers.

**Calculation steps**
1. **Compute cell averages**
   - For each occupied grid cell, take the mean of `price / livingArea`, `livingArea`, and `bedrooms`.
2. **Convert to percentiles**
   - Rank each cell’s value for a metric against all cells (percentile from 0 to 1).
   - This ensures that, for example, a cell in the 90th percentile on `avg_psf` is more expensive than 90% of all cells.
3. **Average percentiles**
   - Take the mean of the three percentile values: `(psf_pct + area_pct + beds_pct) / 3.`
4. **Scale to 0–100**
   - Multiply the mean percentile by 100 to produce a human-readable score.
   - A cell with a score of 80 means it ranks in the top 20% overall across the three factors; a score of 20 means it’s in the bottom 20%.

By using percentile ranks rather than raw values, the composite score balances differences in scale (e.g., dollars vs. square feet) and prevents any single metric with extreme outliers from dominating the index.


## 9. Interactive Google Gemini CLI

This notebook cell lets you chat **directly** with Google Gemini from Colab—no additional backend required. You’ll tap into our real estate–focused assistant and Pinecone property index right from your notebook.

**Before you run this cell:**
- Go to **Colab → Secrets & variables → Notebook secrets** and add:
  - `GOOGLE_API_KEY` = your Google AI API key
  - `PINECONE_API_KEY` = your Pinecone API key
  - `PINECONE_ENVIRONMENT` = your Pinecone environment (e.g. `us-west1-gcp`)
  - `PINECONE_INDEX` = the name of your Pinecone index (e.g. `estatewise-index`)
- Make sure you click **Save** so that `userdata.get()` can retrieve them at runtime.

**How it works:**
1. **Load credentials** from your Colab secrets.
2. **Initialize** the Gemini client and Pinecone index.
3. **Agentic decision**: the assistant will decide if it needs to fetch property embeddings & metadata or just use conversation context.
4. **Expert ensemble**: five specialized “agents” (Data Analyst, Lifestyle Concierge, Financial Advisor, Neighborhood Expert, Cluster Analyst) each produce their view in parallel.
5. A **Master Agent** synthesizes all expert opinions into one cohesive, concise recommendation—always providing at least one property suggestion.
6. **CLI loop**: type your queries at the prompt, get back rich property recommendations, and type `exit` or `quit` to end the session.

_For a full web-app experience with polished UI and persistence, check out our deployed version at_
https://estatewise.vercel.app/


In [ ]:
pip install pinecone google-genai python-dotenv

In [ ]:
import os
import time
import json
import re
import concurrent.futures
from dotenv import load_dotenv
from google import genai
from pinecone import Pinecone
from google.colab import userdata

# 1) Load API keys from Colab secrets
load_dotenv()
api_key = userdata.get('GOOGLE_API_KEY')
if not api_key:
    raise RuntimeError("Set GOOGLE_API_KEY in Colab secrets")
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pinecone_env     = userdata.get('PINECONE_ENVIRONMENT')
pinecone_index   = userdata.get('PINECONE_INDEX')
if not (pinecone_api_key and pinecone_env and pinecone_index):
    raise RuntimeError("Set PINECONE_API_KEY, PINECONE_ENVIRONMENT, and PINECONE_INDEX in Colab secrets")

# 2) Initialize clients
client = genai.Client(api_key=api_key)
pc = Pinecone(api_key=pinecone_api_key, environment=pinecone_env)
index = pc.Index(pinecone_index)

# 3) Pinecone helpers
def sanitize_metadata(md: dict) -> dict:
    out = {}
    for k, v in (md or {}).items():
        if isinstance(v, (str, int, float, bool)):
            out[k] = v
        elif isinstance(v, list):
            out[k] = ", ".join(v) if all(isinstance(x, str) for x in v) else json.dumps(v)
        elif isinstance(v, dict):
            out[k] = json.dumps(v)
        else:
            out[k] = str(v)
    return out

def query_properties(query: str, top_k: int = 30):
    emb = client.embeddings.create(model="models/text-embedding-004", content=query)
    vec = emb.data[0].embedding
    resp = index.query(vector=vec, top_k=top_k, include_metadata=True)
    matches = getattr(resp, "matches", resp.get("matches", []))
    return [
        {
            "id": getattr(m, "id", m.get("id")),
            "score": getattr(m, "score", m.get("score", 0.0)),
            "metadata": sanitize_metadata(getattr(m, "metadata", m.get("metadata", {})))
        }
        for m in matches
    ]

def query_properties_as_string(query: str, top_k: int = 30) -> str:
    props = query_properties(query, top_k)
    if not props:
        return "No matching properties found."
    out = "Matching Properties:\n\n"
    for r in props:
        m = r["metadata"]
        addr = {}
        if "address" in m:
            try: addr = json.loads(m["address"])
            except: addr = {}
        street = addr.get("streetAddress", "Unknown")
        city   = addr.get("city", "Unknown")
        state  = addr.get("state", "Unknown")
        zipc   = addr.get("zipcode", "")
        price  = f"${m['price']}" if m.get("price") else "N/A"
        beds   = m.get("bedrooms", "N/A")
        baths  = m.get("bathrooms", "N/A")
        area   = f"{m['livingArea']} sqft" if m.get("livingArea") else "N/A"
        year   = m.get("yearBuilt", "N/A")
        htype  = m.get("homeType", "N/A")
        desc   = m.get("description", "No description")
        zpid   = str(m.get("zpid", ""))
        link   = f"https://www.zillow.com/homedetails/{zpid}_zpid/" if zpid else "N/A"
        out += (
            f"Property at {street}, {city}, {state} {zipc}\n"
            f"  - Price: {price}\n"
            f"  - Beds: {beds}, Baths: {baths}\n"
            f"  - Living Area: {area}\n"
            f"  - Year Built: {year}\n"
            f"  - Type: {htype}\n"
            f"  - Description: {desc}\n"
            f"  - More details: {link}\n\n"
        )
    return out

# 4) K-Means clustering
def kmeans(data: list[list[float]], k: int, max_iter: int = 20) -> list[int]:
    if not data or k <= 0:
        return []
    n, dims = len(data), len(data[0])
    centroids = [row.copy() for row in data[:min(k, n)]]
    while len(centroids) < k:
        centroids.append(data[-1].copy())
    assign = [0] * n
    for _ in range(max_iter):
        moved = False
        for i, pt in enumerate(data):
            dists = [sum((pt[d] - c[d])**2 for d in range(dims)) for c in centroids]
            c = dists.index(min(dists))
            if assign[i] != c:
                assign[i] = c
                moved = True
        if not moved:
            break
        sums = [[0]*dims for _ in range(k)]
        counts = [0]*k
        for i, pt in enumerate(data):
            c = assign[i]
            counts[c] += 1
            for d in range(dims):
                sums[c][d] += pt[d]
        for c in range(k):
            if counts[c]:
                centroids[c] = [sums[c][d]/counts[c] for d in range(dims)]
    return assign

# 5) Chat function with agentic decision
CLUSTER_COUNT = 4
MAX_HISTORY = 20

def chat_with_estatewise(history: list[str], message: str, user_context: str = "", expert_weights: dict[str,float] = None):
    expert_weights = expert_weights or {}
    start = time.time()
    low = message.strip().lower()

    # 5.0) Greeting / thanks shortcuts
    if low in ("hi", "hello", "hey"):
        return "Hello! How can I assist you today?", {}
    if low in ("thanks", "thank you"):
        return "You're welcome! Let me know if you need anything else.", {}

    # trim history
    hist = history[-MAX_HISTORY*2:]
    hist_str = "\n".join(f"User: {m}" if i%2==0 else f"Assistant: {m}"
                         for i, m in enumerate(hist))

    # 5.1) Agentic decision: should we fetch property data?
    decision_prompt = (
        "You are EstateWise Assistant. First decide whether you need to fetch property data to answer the user. "
        "If the message is small talk (greetings, thanks) or does not ask about properties, answer 'No'. "
        "Otherwise answer 'Yes'.\n\n"
        f"User message:\n\"{message}\"\n\n"
        "Respond with exactly 'Yes' or 'No'."
    )
    decision = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=decision_prompt
    ).text.strip().lower()
    should_fetch = decision.startswith("yes")

    # 5.2) Fetch & cluster if needed
    combined = ""
    if should_fetch:
        try:
            prop_text = query_properties_as_string(message)
            raw = query_properties(message)
        except:
            prop_text, raw = "", []
        if raw:
            vecs = []
            for r in raw:
                m = r["metadata"]
                def to_num(x):
                    s = re.sub(r"[^0-9.\-]", "", str(x) or "0")
                    return float(s) if s else 0.0
                vecs.append([
                    to_num(m.get("price")), to_num(m.get("bedrooms")),
                    to_num(m.get("bathrooms")), to_num(m.get("livingArea")),
                    to_num(m.get("yearBuilt"))
                ])
            dims = len(vecs[0])
            mins = [min(v[i] for v in vecs) for i in range(dims)]
            maxs = [max(v[i] for v in vecs) for i in range(dims)]
            norm = [
                [(v[i]-mins[i])/(maxs[i]-mins[i]) if maxs[i]!=mins[i] else 0.0 for i in range(dims)]
                for v in vecs
            ]
            clusters = kmeans(norm, CLUSTER_COUNT)
            cluster_ctx = "\n".join(f"- Property ID {raw[i]['id']}: cluster {clusters[i]}"
                                    for i in range(len(raw)))
            combined = f"{prop_text}\n\nCluster Assignments:\n{cluster_ctx}"

    # 5.3) Base system instructions
    base_system_instruction = f"""
You are EstateWise Assistant, an expert real estate concierge for Chapel Hill, NC. Provide personalized property recommendations.

Property data & clusters (only if used above):
---------------------------------------------------------
{combined or "None; relying on conversation context only."}
---------------------------------------------------------

When recommending:
1. List address, price, bedrooms, bathrooms, area, year, type.
2. Include description & Zillow link: https://www.zillow.com/homedetails/{{zpid}}_zpid/
3. Numbered list, clear & concise.
4. Use user_context: {user_context or "None"}.
5. Do not ask for more info before recommending.
6. Always give at least one recommendation; never say you cannot.
7. Be concise & conversational.
""".strip()

    # 5.4) Define experts
    experts = [
        {"name":"Data Analyst",        "instr":"Extract stats & trends; be concise."},
        {"name":"Lifestyle Concierge", "instr":"Emphasize lifestyle: schools, parks, commute."},
        {"name":"Financial Advisor",   "instr":"Highlight price trends, mortgage, ROI, taxes."},
        {"name":"Neighborhood Expert", "instr":"Provide safety, walkability, development insights."},
        {"name":"Cluster Analyst",     "instr":f"Summarize the {CLUSTER_COUNT} clusters and key traits."}
    ]

    # normalize weights
    wts = {e["name"]: expert_weights.get(e["name"],1.0) for e in experts}
    total = sum(wts.values()) or len(experts)
    wts = {n: w/total for n,w in wts.items()}

    # 5.5) Call each expert
    expert_out = []
    for e in experts:
        prompt = (
            base_system_instruction + "\n\n" +
            e["instr"] + "\n\n" +
            hist_str + f"\nUser: {message}\nAssistant:"
        )
        resp = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
        expert_out.append({"name":e["name"],"text":resp.text})

    # 5.6) Merge experts
    merged_views = "\n\n".join(
        f"**{r['name']}** (w={wts[r['name']]:.2f}):\n{r['text']}"
        for r in expert_out
    )
    merger_instruction = f"""
You are the EstateWise Master Agent. Synthesize these expert opinions into one cohesive recommendation, following all system instructions above and prioritizing by weight:

{merged_views}
""".strip()

    def do_merge():
        prompt = (
            merger_instruction + "\n\n" +
            base_system_instruction + "\n\n" +
            hist_str + f"\nUser: {message}\nAssistant:"
        )
        return client.models.generate_content(model="gemini-2.0-flash", contents=prompt).text

    # 5.7) Timeout-safe merge
    remaining = 59.0 - (time.time() - start)
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
        fut = ex.submit(do_merge)
        try:
            final = fut.result(timeout=max(0.1, remaining))
        except concurrent.futures.TimeoutError:
            final = max(expert_out, key=lambda r:wts[r["name"]])["text"]

    views = {r["name"]:r["text"] for r in expert_out}
    return final, views

# 6) CLI loop
if __name__=="__main__":
    print("🏡 Welcome to EstateWise CLI! Type 'exit' to quit.\n")
    history = []
    while True:
        msg = input("You: ").strip()
        if msg.lower() in ("exit","quit"):
            print("EstateWise: Goodbye! 👋")
            break
        try:
            reply, _ = chat_with_estatewise(history, msg)
        except Exception as e:
            print(f"Error: {e}")
            break
        print(f"EstateWise: {reply}\n")
        history.extend([msg, reply])
        time.sleep(0.2)
